In [1]:
from __future__ import print_function
from __future__ import division
import torch
import sys
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
import scipy.misc
from sklearn.metrics import confusion_matrix
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import make_grid
import pickle as pkl
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)
import tqdm
from tqdm import tqdm
import pickle

PyTorch Version:  1.5.1
Torchvision Version:  0.6.0a0+35d732a


In [2]:
with open("../weights/simple_fact_q_best_cpu.pkl",'rb') as f:
    Q = pkl.load(f)

In [3]:
with open("../weights/simple_fact_u_best_cpu.pkl",'rb') as f:
    U = pkl.load(f)

In [4]:
class Question_Ans(Dataset):
    def __init__(self, filename, mode='train'):
        self.df = pd.read_csv(filename)
        self.questionid = self.df['QuestionId'].values
        self.userid = self.df['UserId'].values
        self.ans = self.df['IsCorrect'].values
        
#         self.ans = 2*self.ans - 1
        
        self.length=len(self.ans)
        
        
        if(mode=='train'):
            start=int(0*self.length)
            end=int(0.8*self.length)
        elif(mode=='val'):
            start=int(0.8*self.length)
            end=int(1*self.length)
        else:
            start = 0
            end=self.length
            
            
        self.questionid = self.questionid[start:end]
        self.userid = self.userid[start:end]
        self.ans = self.ans[start:end]
        
        self.length=len(self.ans)
        
    def __len__(self):
        return self.length
    
    def __getitem__(self, idx):
        
        qid = self.questionid[idx]
        uid = self.userid[idx]
        ans = self.ans[idx]
        return qid,uid,ans

In [5]:
base_dir = '../Data/'

input_dir = base_dir + "data/train_data"
train_data = base_dir + "data/train_data/train_task_1_2.csv"
test_data_1 = base_dir + "data/test_data/test_public_answers_task_1.csv"

batch_size = 32

In [6]:
# df_test = pd.read_csv(test_data_1)

In [7]:
test_dataset = Question_Ans(filename=test_data_1,mode='test')

In [8]:
test_dataloader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size, shuffle=True)

In [9]:
def get_qvector(questions):
    ans = Q(torch.LongTensor(questions))
    return ans

In [10]:
def get_uvector(users):
    ans = U(torch.LongTensor(users)) 
    return ans

In [11]:
def get_score(qvectors,uvectors):
    q_unsq = torch.unsqueeze(qvectors, 1)
    u_unsq = torch.unsqueeze(uvectors, 2)
    score = torch.bmm(q_unsq,u_unsq)
    score = torch.squeeze(score)
    return score

In [18]:
confusion_matrix = [[0,0],[0,0]]
for questions,users,ans in tqdm(test_dataloader):
    qvectors = get_qvector(questions)
    uvectors = get_uvector(users)
    scores = get_score(qvectors,uvectors)
    
    scores = scores.detach().numpy() + 0
    predictions = np.sign(scores)
    predictions = (predictions + 1)/2
    predictions = predictions.astype(np.int32)
    
    ans = ans.numpy().astype(np.int32)
    
    for i in range(ans.shape[0]):
        confusion_matrix[ans[i]][predictions[i]] += 1


100%|██████████| 61984/61984 [00:11<00:00, 5507.97it/s]


In [19]:
confusion_matrix

[[354364, 353689], [636676, 638752]]